# PyTorch

## Importowanie bibliotek

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
import pandas as pd
from sklearn import metrics
import config_test
import numpy as np

## Wybór urządzenia

In [2]:
device = "cpu" # PyTorch pozwala na wybranie urządzenia, na którym będą przeprowadzane operacje
print(f"Using {device} device")

Using cpu device


## Sieć neuronowa

In [3]:
class NeuralNetwork(nn.Module): 
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(784, 256), # wastwa wejściowa (784 neurony) -> warstwa ukryta (256 neuronów)
            nn.ReLU(), 
            nn.Linear(256, 256), # warstwa ukryta (256 neuronów) -> warstwa ukryta (256 neuronów)
            nn.ReLU(),
            nn.Linear(256, 10), # warstwa ukryta (256 neuronów) -> wartswa wyjściowa (10 neuronów) 
        )

    def forward(self, x): # przesuwamy się z warstwy do warstwy
        logits = self.linear_relu_stack(x)
        return logits

## Dostosowanie danych, trenowanie i testowanie modelu

In [4]:
df = pd.read_csv(config_test.NEW_DATA_PATH) #jednym z lepszych sposobów na wrzucenie danych do tensorów jest użycie dataframe

# Przechodzimy po kolei po wszystkich foldach
for fold in range(config_test.FOLDS_COUNT):
    

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~inicjalizacja danych~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    torch_model = NeuralNetwork().to(device) # model
    
    # dzielimy nasze dane na treningowe i testowe
    df_train = df.loc[df['kfold'] != fold] # dane treningowe = dane, które nie należą do obecnego folda
    df_test = df.loc[df['kfold'] == fold] # dane testowe = dane, które należą do obecnego folda

    # wypisujemy liczbę wierszy i kolumn
    # liczba wierszy powinna wynosić łącznie 10000, bo tyle mamy obrazków
    # liczba kolumn powinna wynosić łącznie 28*28 + 2 (liczba pikseli jednego obrazka + kolumna label i kolumna kfold)
    #print(df_train.shape)
    #print(df_test.shape)

    # zamieniamy dataframe na tensory
    t_train = torch.tensor(df_train.values, dtype=torch.float32)
    t_test = torch.tensor(df_test.values, dtype=torch.float32)
    
    # dzielimy dane na dane bez labeli i labele
    # żeby przeprowadzanie operacji na tensorach było możliwe, to dane bez labeli muszą być zmiennoprzecinkowe, a labele całkowite
    t_train_label = t_train[:,-2:-1]
    t_train = t_train[:,:-2] # tensor treningowy zawierający 28*28 kolumn
    t_train_label = t_train_label.long() # tensor treningowy zawierający kolumnę label
    #print(t_train.shape)
    #print(t_train_label.shape)

    t_test_label = t_test[:,-2:-1]
    t_test = t_test[:,:-2] # tensor testowy zawierający 28*28 kolumn
    t_test_label = t_test_label.long() # tensor testowy zawierający kolumnę label
    #print(t_test.shape)
    #print(t_test_label.shape)
    
    # DataLoader ułatwia dostęp do danych, ładuje dane w określonych partiach (batchach) i losowo miesza dane w każdej epoce
    train_dataset = TensorDataset(t_train, t_train_label) # Dataset łączy dane dla DataLoadera
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    test_dataset = TensorDataset(t_test, t_test_label)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

    # przenosimy tensory do urządzenia
    t_train, t_train_label = t_train.to(device), t_train_label.to(device)
    t_test, t_test_label = t_test.to(device), t_test_label.to(device)

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~koniec inicjalizacji danych~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~trenowanie danych~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    criterion = nn.CrossEntropyLoss()  # Funkcja kosztu dla klasyfikacji
    optimizer = torch.optim.Adam(torch_model.parameters(), lr=0.001)  # Dostosowuje parametry modelu

    # ustawiamy ziarno generatora liczb losowych, losowe funkcje w PyTorch będą się zachowywać deterministycznie
    torch.manual_seed(42)

    # liczba epok
    epochs = 10

    # pętla treningowa
    for epoch in range(epochs):

        # ustawienie modelu w tryb treningowy
        torch_model.train()

        # oblicza straty, które powstały w trakcie trenowania modelu
        running_loss = 0.0

        for X_batch, y_batch in train_loader: #X_batch - batch z t_train, y_batch - batch z t_train_label
            logits = torch_model(X_batch) # przekazujemy batch danych z t_train do modelu i otrzymujemy logity
            y_batch = y_batch.squeeze(1) # usuwamy zbędny wymiar (konieczne dla następnej funkcji)
            loss = criterion(logits, y_batch) # obliczamy stratę pomiędzy przewidywaniami modelu a labels

            # czyszczenie poprzednich gradientów
            optimizer.zero_grad()

            # backpropagation
            loss.backward()

            # aktualizacja wag
            optimizer.step()

            # do strat dodajemy wartość obliczoną przez criterion
            running_loss += loss.item()
        

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~testowanie danych ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        # ustawienie modelu w tryb oceny
        torch_model.eval()
        correct = 0 # poprawnie odgadnięte cyfry
        total = 0 # wszystkie cyfry
        val_loss = 0 # wartość straty

        with torch.no_grad(): # wyłączenie liczenia gradientów
            for X_val, y_val in test_loader: # X_val - batch z t_test, y_val - batch z t_test_label 
                logits = torch_model(X_val) # przekazujemy batch danych z t_test do modelu i otrzymujemy logity
                y_val = y_val.squeeze(1) # usuwamy dimensions o rozmiarze 1 (konieczne dla następnej funkcji)
                loss = criterion(logits, y_val) # obliczamy stratę pomiędzy przewidywaniami modelu a labels
                val_loss += loss.item() # do strat dodajemy wartość obliczoną przez criterion

                pred = logits.argmax(dim=1) # uzyskujemy indeks klasy przewidywanej
                correct += (pred == y_val).sum().item() # sumujemy liczbę poprawnych przewidywań
                total += y_val.size(0) #zwiększamy całkowitą liczbę przykładów

        accuracy = correct / total

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}, Accuracy: {accuracy}")



Epoch 1/10, Loss: 1.0163910608887672, Accuracy: 0.9025
Epoch 2/10, Loss: 0.17150246965885163, Accuracy: 0.9335
Epoch 3/10, Loss: 0.10580987068265676, Accuracy: 0.936
Epoch 4/10, Loss: 0.07617094045504928, Accuracy: 0.9325
Epoch 5/10, Loss: 0.0908170979609713, Accuracy: 0.945
Epoch 6/10, Loss: 0.06007138892263174, Accuracy: 0.937
Epoch 7/10, Loss: 0.0736413908386603, Accuracy: 0.9415
Epoch 8/10, Loss: 0.07056491583457682, Accuracy: 0.9415
Epoch 9/10, Loss: 0.07431045561283826, Accuracy: 0.9345
Epoch 10/10, Loss: 0.07307876492175273, Accuracy: 0.9445
Epoch 1/10, Loss: 1.0571453796625137, Accuracy: 0.922
Epoch 2/10, Loss: 0.15037720242142677, Accuracy: 0.9325
Epoch 3/10, Loss: 0.10363032282143832, Accuracy: 0.9335
Epoch 4/10, Loss: 0.08843611774593592, Accuracy: 0.9295
Epoch 5/10, Loss: 0.09193523762747645, Accuracy: 0.9405
Epoch 6/10, Loss: 0.0364772818335332, Accuracy: 0.95
Epoch 7/10, Loss: 0.03916223005065694, Accuracy: 0.9405
Epoch 8/10, Loss: 0.0907833192194812, Accuracy: 0.935
Epoc